<a href="https://colab.research.google.com/github/drmartins2/EDIT_DE/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00


# Producer

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(60)


False

In [4]:
query.stop()

In [5]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/*")
df.show()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-14 14:43:...|    0|      SENT|76a798a2-63cf-4bc...|    SMS|      2001|   1046|
|2024-12-14 14:43:...|    2|      SENT|76a798a2-63cf-4bc...|    SMS|      2001|   1046|
|2024-12-14 14:43:...|    1|      SENT|76a798a2-63cf-4bc...|    SMS|      2001|   1046|
|2024-12-14 14:43:...|    3|      SENT|76a798a2-63cf-4bc...|    SMS|      2001|   1046|
|2024-12-14 14:44:...|   57|  RECEIVED|f0d2968a-d593-4e5...|  EMAIL|      2003|   1028|
|2024-12-14 14:44:...|   61|  RECEIVED|936f7295-3051-492...|  EMAIL|      2013|   1011|
|2024-12-14 14:44:...|   42|  RECEIVED|b424a48f-c50d-4ed...|  EMAIL|      2008|   1034|
|2024-12-14 14:45:...|   69|   CLICKED|0db88870-ec0b-48e...|  EMAIL|      2006|   1025|
|2024-12-14 14:44:...|   59|  RE

#New Producer

In [9]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession
import shutil
import os

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
    fake = Faker()
    new_columns = {
        'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
        'message_id': F.lit(fake.random_element(elements=messages)),
        'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
        'country_id': F.lit(fake.random_int(min=2000, max=2015)),
        'user_id': F.lit(fake.random_int(min=1000, max=1050)),
    }
    df = df.withColumns(new_columns)
    return df

def insert_messages(df: DataFrame, batch_id):
    enrich = enrich_data(df)
    enrich = enrich.withColumn("value", F.col("value").cast("string"))
    enrich.write.mode("append").format("parquet").save("/content/lake/bronze/messages/data")

# Delete existing data - For first execution there's no data to delete (as directory is different from the other Producer)
bronze_path = "/content/lake/bronze/messages/data"
if os.path.exists(bronze_path):
    shutil.rmtree(bronze_path)

# Read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write stream
query = (df_stream.writeStream
    .outputMode('append')
    .trigger(processingTime='1 seconds')
    .option("checkpointLocation", "/content/lake/bronze/messages/checkpoint")
    .foreachBatch(insert_messages)
    .start()
)

# Run for 1 minute
query.awaitTermination(60)


StreamingQueryException: [STREAM_FAILED] Query [id = e621e8bb-81a2-42e2-bf61-33f5d63fd0b2, runId = 64d1b5b1-bc94-4163-9a3d-60c074406365] terminated with exception: [INTERNAL_ERROR] The Spark SQL phase planning failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.

In [ ]:
query.stop()

In [11]:
# Check New Producer results
df = spark.read.format("parquet").load("/content/lake/bronze/messages/data")
print("Total messages in bronze layer:", df.count())

df.filter(~((F.col("event_type").isNull()) | (F.col("event_type") == "") | (F.col("event_type") == "NONE"))).show()

Total messages in bronze layer: 58
+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-14 09:32:...|   48|  RECEIVED|a3e6fe00-39d0-44a...|  OTHER|      2003|   1028|
|2024-12-14 09:31:...|   12|  RECEIVED|d3ed673c-69f2-4bf...|  EMAIL|      2006|   1000|
|2024-12-14 09:31:...|   24|   CREATED|4b89e6d8-001d-423...|  EMAIL|      2007|   1003|
|2024-12-14 09:32:...|   36|   CLICKED|04cd390c-9060-44d...|  EMAIL|      2009|   1029|
|2024-12-14 09:32:...|   46|  RECEIVED|a8861a70-9db8-48d...|   PUSH|      2000|   1021|
|2024-12-14 09:32:...|   49|  RECEIVED|2a9c309b-fa95-40e...|   PUSH|      2009|   1007|
|2024-12-14 09:31:...|   10|  RECEIVED|c8f39c1d-b8e0-420...|   PUSH|      2015|   1035|
|2024-12-14 09:31:...|   21|   CREATED|daa7a8be-040b-41c...|  OTHER|      2007|   105

# Additional datasets
[Deprecated]

In [12]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, date_format, expr
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

spark = SparkSession.builder.appName("MessageProcessing").getOrCreate()

# Define messages schema
message_schema = StructType([
    StructField("timestamp", TimestampType()),
    StructField("value", StringType()),
    StructField("event_type", StringType()),
    StructField("message_id", StringType()),
    StructField("channel", StringType()),
    StructField("country_id", IntegerType()),
    StructField("user_id", IntegerType())
])

# Added countries data to this cell for join optimization by caching countries dataset
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries).cache()

def process_messages(df, batch_id):
    try:
        logger.info(f"Processing batch {batch_id}")

        # Join with countries
        df_with_country = df.join(countries, "country_id", "left")

        # Add date column
        df_with_date = df_with_country.withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))

        # Split into corrupted and non-corrupted
        valid = df_with_date.filter((col("event_type").isNotNull()) & (col("event_type") != "") & (col("event_type") != "NONE"))


        # Write non-corrupted data
        valid.write \
            .partitionBy("date") \
            .mode("append") \
            .parquet("/content/lake/silver/messages/data")

        logger.info(f"Batch {batch_id} processed successfully")
    except Exception as e:
        logger.error(f"Error processing batch {batch_id}: {str(e)}")


def process_messages_corrupted(df, batch_id):
    try:
        logger.info(f"Processing batch {batch_id}")

        # Join with countries
        df_with_country = df.join(countries, "country_id", "left")

        # Add date column
        df_with_date = df_with_country.withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))

        # Split into corrupted and non-corrupted
        corrupted = df_with_date.filter((col("event_type").isNull()) | (col("event_type") == "") | (col("event_type") == "NONE"))

        # Write corrupted data
        corrupted.write \
            .partitionBy("date") \
            .mode("append") \
            .parquet("/content/lake/silver/messages_corrupted/data")

        logger.info(f"Batch {batch_id} processed successfully")
    except Exception as e:
        logger.error(f"Error processing batch {batch_id}: {str(e)}")


# Read from bronze layer
df_stream = spark.readStream \
    .schema(message_schema) \
    .parquet("/content/lake/bronze/messages/data/*")

# Process stream
query = df_stream.writeStream \
    .foreachBatch(process_messages) \
    .option("checkpointLocation", "/content/lake/silver/checkpoint") \
    .trigger(processingTime='5 seconds') \
    .start()

query2 = df_stream.writeStream \
    .foreachBatch(process_messages_corrupted) \
    .option("checkpointLocation", "/content/lake/silver/checkpoint_corrupted") \
    .trigger(processingTime='5 seconds') \
    .start()

# Run for 20 seconds
query.awaitTermination(20)
query.stop()

query2.awaitTermination(20)
query2.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, date_format, expr
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

spark = SparkSession.builder.appName("MessageProcessing").getOrCreate()

# Define messages schema
message_schema = StructType([
    StructField("timestamp", TimestampType()),
    StructField("value", StringType()),
    StructField("event_type", StringType()),
    StructField("message_id", StringType()),
    StructField("channel", StringType()),
    StructField("country_id", IntegerType()),
    StructField("user_id", IntegerType())
])

# Added countries data to this cell for join optimization by caching countries dataset
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries).cache()

def process_messages(df, batch_id):
    try:
        logger.info(f"Processing batch {batch_id}")

        # Join with countries
        df_with_country = df.join(countries, "country_id", "left")

        # Add date column
        df_with_date = df_with_country.withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))

        # Split into corrupted and non-corrupted with corresponding logic
        corrupted = df_with_date.filter((col("event_type").isNull()) | (col("event_type") == "") | (col("event_type") == "NONE"))
        valid = df_with_date.filter((col("event_type").isNotNull()) & (col("event_type") != "") & (col("event_type") != "NONE"))


        # Write corrupted data
        corrupted.write \
            .partitionBy("date") \
            .mode("append") \
            .parquet("/content/lake/silver/messages_corrupted/data")

        # Write non-corrupted data
        valid.write \
            .partitionBy("date") \
            .mode("append") \
            .parquet("/content/lake/silver/messages/data")


        logger.info(f"Batch {batch_id} processed successfully")
    except Exception as e:
        logger.error(f"Error processing batch {batch_id}: {str(e)}")


# Read from bronze layer
df_stream = spark.readStream \
    .schema(message_schema) \
    .parquet("/content/lake/bronze/messages/data/*")

# Process stream
query = df_stream.writeStream \
    .foreachBatch(process_messages) \
    .option("checkpointLocation", "/content/lake/silver/checkpoint") \
    .trigger(processingTime='5 seconds') \
    .start()

# Run for 20 seconds
query.awaitTermination(20)
query.stop()


## Checking data

In [ ]:
# Check results
bronze_count = spark.read.schema(message_schema).parquet("content/lake/bronze/messages/data").count()
silver_corrupted_count = spark.read.parquet("content/lake/silver/messages_corrupted/data").count()
silver_valid_count = spark.read.parquet("content/lake/silver/messages/data").count()

logger.info(f"Total messages in bronze layer: {bronze_count}")
logger.info(f"Total corrupted messages in silver layer: {silver_corrupted_count}")
logger.info(f"Total non-corrupted messages in silver layer: {silver_valid_count}")
logger.info(f"Total messages in silver layer: {silver_corrupted_count + silver_valid_count}")

assert bronze_count == silver_corrupted_count + silver_valid_count, "Mismatch in message counts"

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [ ]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [ ]:
# report 1
# TODO

## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [ ]:
# report 2
# TODO

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

